In [1]:
import os
import sys

notebook_dir = os.getcwd()
project_root_path = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root_path)

from src.models import ModelXtoCResNet
from src.preprocessing.RIVAL10 import preprocessing_rival10
from src.utils import *
from src.training import run_epoch_x_to_c

In [ ]:
concept_labels, train_loader, test_loader = preprocessing_rival10(training=True, class_concepts=True, verbose=True)

Found 26384 unique images.
Found 18 unique concepts.
Generated one-hot training matrix with shape: (21098, 10)
Found 21098 images.
Processing in 330 batches of size 64 (for progress reporting)...


Processing batches: 100%|█████████████████████| 330/330 [00:36<00:00,  9.07it/s]



Finished processing.
Successfully transformed: 21098 images.
Found 5286 images.
Processing in 83 batches of size 64 (for progress reporting)...


Processing batches: 100%|███████████████████████| 83/83 [00:13<00:00,  6.31it/s]



Finished processing.
Successfully transformed: 5286 images.
Dataset initialized with 21098 pre-sorted items.
Dataset initialized with 5286 pre-sorted items.


# Training Implementation

In [ ]:
from src.utils import find_class_imbalance
from src.config import RIVAL10_CONFIG as config_dict
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
N_TRIMMED_CONCEPTS = config_dict['N_TRIMMED_CONCEPTS']
N_CLASSES = config_dict['N_CLASSES']

**Find device to run model on (CPU or GPU).**

In [5]:
device = torch.device("cuda" if torch.cuda.is_available()
                    else "mps" if torch.backends.mps.is_available()
                    else "cpu")
print(f"Using device: {device}")

Using device: mps


**Instantiate the model.**

In [6]:
model = ModelXtoCResNet(pretrained=True,
                freeze=True,
                n_concepts=N_TRIMMED_CONCEPTS)

model = model.to(device)
print("Model Instantiated (X -> C)")

/Users/pb/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/pb/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model Instantiated (X -> C)


### Loss
We use weighted loss.

`BCEWithLogitsLoss()` performs 2 steps:
1. $\sigma(x)$
    - Applies the sigmoid function to the logits to get probabilities.
2. $\text{BCE}(\sigma(x), y) = y \cdot \text{log}(\sigma(x)) + (1-y) \cdot (1-\text{log}(\sigma(x)))$
    - Compute binary cross-entropy between output probabilities ($\sigma(x)$) and ground truths ($y$)

In [7]:
use_weighted_loss = True # Set to False for simple unweighted loss

if use_weighted_loss:
    concept_weights = find_class_imbalance(concept_labels)
    attr_criterion = [nn.BCEWithLogitsLoss(weight=torch.tensor([ratio], device=device, dtype=torch.float))
                    for ratio in concept_weights]
else:
    attr_criterion = [nn.BCEWithLogitsLoss() for _ in range(N_TRIMMED_CONCEPTS)]

### Optimiser
Use same settings as used in CBM repo.

In [8]:
lr = 0.01
weight_decay = 0.00004 # same as lambda in L2-regularisation

optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                    lr=lr,
                    momentum=0.9,
                    weight_decay=weight_decay)

# scheduler_step = n -> decrease the LR every n epochs
scheduler_step = 1000
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=0.1)

print("Optimizer and Scheduler Ready")

Optimizer and Scheduler Ready


### Training and Validation Loops

In [9]:
epochs = 50
log_interval = 50

best_val_acc = 0.0

In [10]:
for epoch in range(epochs):
    print(f"--- Epoch {epoch+1}/{epochs} ---")

    # Train
    train_loss, train_acc = run_epoch_x_to_c(model, train_loader, attr_criterion, optimizer, is_training=True, use_aux=True, n_concepts=N_TRIMMED_CONCEPTS, device=device, verbose=True)
    print(f'Epoch {epoch+1} Train Summary | Loss: {train_loss:.4f} | Acc: {train_acc:.3f}')

    # Validate
    if test_loader:
        with torch.no_grad():
            val_loss, val_acc = run_epoch_x_to_c(model, test_loader, attr_criterion, optimizer, n_concepts=N_TRIMMED_CONCEPTS, device=device, verbose=True)

        print(f'Epoch {epoch+1} Val Summary   | Loss: {val_loss:.4f} | Acc: {val_acc:.3f}')

        # Save best model based on validation accuracy
        if val_acc > best_val_acc:
            print(f"Validation accuracy improved ({best_val_acc:.3f} -> {val_acc:.3f}). Saving model...")
            best_val_acc = val_acc
            torch.save(model, 'x_to_c_best_model.pth')
            print("Model saved to x_to_c_best_model.pth")

    # Scheduler step
    scheduler.step()
    print(f"Current LR: {optimizer.param_groups[0]['lr']}")

--- Epoch 1/50 ---


Epoch 1 Train Summary | Loss: 0.3839 | Acc: 96.637


Validation:   0%|                                        | 0/83 [00:00<?, ?it/s]python3.11(90369) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(90377) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(90378) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(90379) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
                                                                                

Epoch 1 Val Summary   | Loss: 0.1848 | Acc: 98.576
Validation accuracy improved (0.000 -> 98.576). Saving model...
Model saved to x_to_c_best_model.pth
Current LR: 0.01
--- Epoch 2/50 ---


Training:   0%|                                         | 0/330 [00:00<?, ?it/s]python3.11(90436) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(90437) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(90438) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(90439) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
                                                                                

...

Epoch 12 Train Summary | Loss: 0.1244 | Acc: 98.820


Validation:   0%|                                        | 0/83 [00:00<?, ?it/s]python3.11(92100) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(92105) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(92108) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(92109) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
                                                                                

Epoch 12 Val Summary   | Loss: 0.1253 | Acc: 98.928
Current LR: 0.01
--- Epoch 13/50 ---


Training:   0%|                                         | 0/330 [00:00<?, ?it/s]python3.11(92158) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(92159) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(92160) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.11(92161) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
                                                                                

KeyboardInterrupt: 

In [ ]:
if test_loader:
    with torch.no_grad():

        test_loss, test_acc, outputs = run_epoch_x_to_c(
            model, test_loader, attr_criterion, optimizer=None, n_concepts=N_TRIMMED_CONCEPTS, device=device,
            return_outputs='sigmoid', verbose=True
        )

# print(f"Shuffled labels shape: {shuffled_img_labels.shape}")
print(f'Best Model Summary   | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')